In [ ]:
# imports

import os
import re
import math
import random
import numpy as np
from typing import Optional
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from huggingface_hub import login
from tqdm import tqdm
import matplotlib.pyplot as plt
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoTokenizer

In [ ]:
# environment

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [ ]:
# constants

MIN_TOKENS = 80
MAX_TOKENS = 180
CUTOFF_CHARS = MAX_TOKENS * 7

In [ ]:
# initialize

openai = OpenAI()
claude = anthropic.Anthropic()
OPENAI_MODEL = "gpt-4o-mini"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"
hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

In [ ]:
%matplotlib inline

In [ ]:
# Constants

BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# Used for writing to output in color

GREEN = "\033[92m"
YELLOW = "\033[93m"
RED = "\033[91m"
RESET = "\033[0m"

In [ ]:
datasets = ["Electronics", "Appliances", "Cell_Phones_and_Accessories", "Home_and_Kitchen", "Tools_and_Home_Improvement"]

In [ ]:
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_Electronics", split="full", trust_remote_code=True)

In [ ]:
print(f"There are {len(dataset):,} items in the dataset")
print("Here is the first:")
item = dataset[0]
print(item['title'])
print(item['description'])
print(item['features'])
print(item['price'])

In [ ]:
class Item:
    
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
    PREFIX = "Price is $"

    title: str
    price: float
    token_count: int = 0
    details: Optional[str]
    prompt: Optional[str]

    def __init__(self, data, price):
        self.title = data['title']
        self.price = price
        self.create_details(data)
        
    def create_details(self, data):
        self.details = '\n'.join(data['description'])
        features = '\n'.join(data['features'])
        if features:
            self.details += '\n' + features
        self.details = re.sub(r'[\[\]【】\s]+', ' ', self.details).strip()
        self.make_prompt()

    def question(self):
        prompt = "How much does this cost?\n"
        prompt += f"Title: {self.title}\n"
        prompt += f"Details: {self.details}\n"
        return prompt

    def messages(self):
        return [
            {"role":"system", "content": "You estimate product prices. Reply only with the price to the nearest dollar"},
            {"role":"user", "content": self.question()},
            {"role":"assistant", "content": f"{self.PREFIX}{str(round(self.price))}.00"}
        ]

    def make_prompt(self):
        prompt = self.tokenizer.apply_chat_template(self.messages(), tokenize=False, add_generation_prompt=False)
        groups = prompt.split('\n\n')
        self.prompt = groups[0]+'\n\n'+'\n\n'.join(groups[2:])

    def count_tokens(self):
        self.token_count = len(self.tokenizer.encode(self.prompt))

    def tokens_between(self, low, high):
        return self.token_count >= low and self.token_count < high

    def test_prompt(self):
        return self.prompt.split(self.PREFIX)[0] + self.PREFIX

In [ ]:
def read_dataset(name):
    print(f"Loading dataset {name}")
    dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", f"raw_meta_{name}", split="full", trust_remote_code=True)
    results = []
    for data in tqdm(dataset):
        try:
            price_str = data['price']
            if price_str:
                price = float(price_str)
                if price > 0:
                    results.append(Item(data, price))
        except ValueError:
            pass
    print(f"Completed loading {name} with {len(results):,} datapoints")
    return results

In [ ]:
items = []
for dataset in datasets:
    items.extend(read_dataset(dataset))

In [ ]:
# Let's investigate:

print(f"There are {len(items):,} items with prices\n")
print(items[0].prompt)
print(items[1].prompt)

In [ ]:
# Plot the distribution of character count

lengths = [len(item.prompt) for item in items]
fig, ax = plt.subplots(1, 1)
ax.set_xlabel('Length')
ax.set_ylabel('Count of items');
_ = ax.hist(lengths, rwidth=0.7, color="lightblue", bins=range(0, 5000, 250))

print(f"Average length is {sum(lengths)/len(lengths):,.1f} and highest length is {max(lengths):,}\n")

In [ ]:
print(f"There are total {len(items):,} items")
selection = [item for item in items if len(item.prompt) < CUTOFF_CHARS]
print(f"There are total {len(selection):,} with under {CUTOFF_CHARS:,} character training prompt")

In [ ]:
# Calculate token sizes in selection

for item in tqdm(selection):
    item.count_tokens()

In [ ]:
# Plot the distribution of tokens

token_counts = [item.token_count for item in tqdm(selection)]
fig, ax = plt.subplots(1, 1)
ax.set_xlabel('Number of tokens')
ax.set_ylabel('Count of items');
_ = ax.hist(token_counts, rwidth=0.7, color="orange", bins=range(0, 500, 25))

In [ ]:
def report(item):
    prompt = item.prompt
    tokens = Item.tokenizer.encode(item.prompt)
    print(prompt)
    print(tokens[-10:])
    print(Item.tokenizer.batch_decode(tokens[-10:]))

In [ ]:
report(items[0])

In [ ]:
# Let's limit our dataset to documents with 60-180 tokens

subset = [item for item in tqdm(selection) if item.tokens_between(MIN_TOKENS, MAX_TOKENS)]
subset_count = len(subset)
count = len(items)
print(f"\nBetween {MIN_TOKENS} and {MAX_TOKENS}, we get {subset_count:,} out of {count:,} which is {subset_count/count*100:.1f}%")

In [ ]:
# Plot the distribution again to check it looks as expected

token_counts = [item.token_count for item in subset]
fig, ax = plt.subplots(1, 1)
ax.set_xlabel('Number of tokens')
ax.set_ylabel('Count of items');
_ = ax.hist(token_counts, rwidth=0.7, color="purple", bins=range(0, 300, 10))

In [ ]:
# Plot the distribution of prices

prices = [float(item.price) for item in subset]
fig, ax = plt.subplots(1, 1)
ax.set_xlabel('Price ($)')
ax.set_ylabel('Count of items');
_ = ax.hist(prices, rwidth=0.7, color="darkblue", bins=range(0, 500, 20))

print(f"Average price is ${sum(prices)/len(prices):.2f} and highest price is ${max(prices):,.2f}\n")

In [ ]:
# Pick the most expensive 52,000 items, then pick 12,000 of the next 20,000

# random.seed(42)
# subset2 = [item for item in subset if item.price <= 999]
# sorted_subset2 = sorted(subset2, key=lambda item: item.price, reverse=True)
# sample = sorted_subset2[:90_000]
# other_12k = random.sample(sorted_subset2[90_000:130_000], k=15000)
# sample += other_12k
# print(f"Created a sample of {len(sample):,} with prices ranging from ${sample[-1].price} to ${sample[0].price}")

In [ ]:
random.seed(42)
subset = [item for item in subset if item.price <= 999]
sorted_subset = sorted(subset, key=lambda item: item.price, reverse=True)
sample = sorted_subset[:150_000]
sample += random.sample(sorted_subset[150_000:300_000], k=50000)
sample += random.sample(sorted_subset[300_000:], k=5000)
print(f"Created a sample of {len(sample):,} with prices ranging from ${sample[-1].price} to ${sample[0].price}")

In [ ]:
# Plot the distribution of prices

plt.figure(figsize=(10, 6))
prices = [float(item.price) for item in sample]
plt.hist(prices, rwidth=0.7, color="orange", bins=range(0, 1000, 20))

plt.title(f"Avg price ${sum(prices)/len(prices):.2f}")
plt.xlabel('Price ($)')
plt.ylabel('Count of items')

plt.show()

In [ ]:
sizes = [len(item.prompt) for item in sample]
prices = [item.price for item in sample]

# Create the scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(sizes, prices, s=1, color="red")

# Add labels and title
plt.xlabel('Size')
plt.ylabel('Price')
plt.title('Is there a simple correlation?')

# Display the plot
plt.show()

In [ ]:
# Plot the distribution again to check it looks as expected

token_counts = [item.token_count for item in sample]
fig, ax = plt.subplots(1, 1)
ax.set_xlabel('Number of tokens')
ax.set_ylabel('Count of items');
_ = ax.hist(token_counts, rwidth=0.7, color="purple", bins=range(0, 300, 10))

In [ ]:
report(sample[-1])

In [ ]:
random.seed(42)
random.shuffle(sample)
train = sample[:200_000]
test = sample[200_000:]
print(f"Divided into a training set of {len(train):,} items and test set of {len(test):,} items")

In [ ]:
del items, subset, sorted_subset, selection

In [ ]:
test[0].title

In [ ]:
import csv
with open('test.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    for t in test[:200]:
        writer.writerow([t.title, t.details, 0])

In [ ]:
human_predictions = []
with open('human.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        human_predictions.append(float(row[2]))

In [ ]:
average = sum(t.price for t in train)/len(train)
average

In [ ]:
class TestRunner:

    def __init__(self, predictor, data, title, size=200):
        self.predictor = predictor
        self.data = data
        self.title = title
        self.size = size
        self.guesses = []
        self.truths = []
        self.errors = []
        self.sles = []
        self.colors = []

    def run_datapoint(self, i):
        datapoint = self.data[i]
        guess = self.predictor(datapoint)
        truth = datapoint.price
        error = abs(guess - truth)
        log_error = math.log(truth+1) - math.log(guess+1)
        sle = log_error ** 2
        color = RED if error>=20 else YELLOW if error>=10 else GREEN
        color_str = "red" if error>=20 else "yellow" if error>=10 else "green"
        title = datapoint.title if len(datapoint.title) <= 40 else datapoint.title[:40]+"..."
        self.guesses.append(guess)
        self.truths.append(truth)
        self.errors.append(error)
        self.sles.append(sle)
        self.colors.append(color_str)
        print(f"{color}{i+1}: Guess: ${guess:,.2f} Truth: ${truth:,.2f} Error: ${error:,.2f} SLE: {sle:,.2f} Item: {title}{RESET}")

    def chart(self, title):
        max_error = max(self.errors)
        plt.figure(figsize=(12, 8))
        plt.scatter(self.truths, self.guesses, s=3, c=self.colors)
        plt.xlabel('Ground Truth')
        plt.ylabel('Model Estimate')
        plt.title(title)
        plt.show()

    def report(self):
        average_error = sum(self.errors) / self.size
        rmsle = math.sqrt(sum(self.sles) / self.size)
        hits = [e for e in self.errors if e<10]
        title = f"{self.title} Error=${average_error:,.2f} RMSLE={rmsle:,.2f} Hits={len(hits)/self.size*100:.1f}%"
        self.chart(title)

    def run(self):
        self.error = 0
        for i in range(self.size):
            self.run_datapoint(i)
        self.report()
        return self

In [ ]:
train_average = sum(t.price for t in train)/len(train)

def flat_predictor(item):
    return train_average

In [ ]:
runner = TestRunner(flat_predictor, test, "Flat Predictor").run()

In [ ]:
human_predictions = []
with open('human.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        human_predictions.append(float(row[2]))

def human_predictor(item):
    index = test.index(item)
    if index==-1:
        raise ValueError("Index not found")
    return human_predictions[index]

In [ ]:
runner = TestRunner(human_predictor, test, "Human Predictor").run()

In [ ]:
stop = set(['the', 'and', 'for', 'is', 'to', 'this', 'with', 'a', 'of', 'your', 'are', 'in','from', 'you', 'or', 'an', 'on', 'by'])

def words(item):
  text = f"{item.title} {item.details}"
  text = re.sub(r'[\(\)\[\]\{\},\'"\- \s]+', ' ', text)
  words = text.strip().lower().split(' ')
  filtered = [word for word in words if word not in stop]
  return " ".join(filtered)

In [ ]:
documents = [words(item) for item in train]
from collections import Counter
count = Counter()
for doc in documents:
    ws = doc.split(" ")
    for w in ws:
        count[w]+=1
count.most_common(30)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression
import numpy as np

np.random.seed(42)

labels = np.array([float(item.price) for item in train])

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(documents)

regressor = LinearRegression()
regressor.fit(X, labels)

In [ ]:
def linear_regression_predictor(item):
    np.random.seed(42)
    x = vectorizer.transform([words(item)])
    return max(regressor.predict(x)[0], 0)

In [ ]:
TestRunner(linear_regression_predictor, test, "Linear Regression", 200).run()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVR

np.random.seed(42)

documents = [words(item) for item in train]
labels = np.array([float(item.price) for item in train])

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(documents)

regressor = SVR(kernel='linear')
regressor.fit(X, labels)

In [ ]:
def svr_predictor(item):
    np.random.seed(42)
    x = vectorizer.transform([words(item)])
    return max(regressor.predict(x)[0], 0)

In [ ]:
TestRunner(svr_predictor, test, "SVR Accuracy", 200).run()

In [ ]:
def messages_for(item):
    system_message = "You estimate product prices. Reply only with the price, no explanation"
    user_prompt = item.question()
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt}
    ]

In [ ]:
def get_price(s):
    s = s.replace('$','').replace(',','')
    match = re.search(r"[-+]?\d*\.\d+|\d+", s)
    return float(match.group()) if match else 0

In [ ]:
def gpt_predictor(item):
    response = openai.chat.completions.create(
        model=OPENAI_MODEL,
        messages=messages_for(item),
        seed=42,
        max_tokens=8
    )
    reply = response.choices[0].message.content
    return get_price(reply)

In [ ]:
runner = TestRunner(gpt_predictor, test, "GPT-4o Prediction Accuracy", 200).run()

In [ ]:
report(train[1])

In [ ]:
train_prompts = [item.prompt for item in train]
train_prices = [item.price for item in train]
test_prompts = [item.test_prompt() for item in test]
test_prices = [item.price for item in test]

In [ ]:
test_prompts[1]

In [ ]:
# Create a Dataset from the lists
train_dataset = Dataset.from_dict({"text": train_prompts, "price": train_prices})
test_dataset = Dataset.from_dict({"text": test_prompts, "price": test_prices})
dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

In [ ]:
DATASET_NAME = "ed-donner/multi-instruct"
dataset.push_to_hub(DATASET_NAME, private=True)